In [1]:
import numpy as np
from scipy.fftpack import dct
import scipy
from functools import lru_cache

### Zadanie 1

1. Zaimplementuj funkcję realizującą DFT jako iloczyn macierzy Fouriera Fn i nelementowego wektora wejściowego (y = Fn @ x).

In [2]:
def dzeta(n):
    return lambda jk: np.cos(2*np.pi*jk/n) - 1.j*np.sin(2*np.pi*jk/n)
def F(n):
    jk_matrix = np.arange(n).reshape(-1,1) @ np.arange(n).reshape(1,-1)
    return dzeta(n)(jk_matrix)
def dft(x):
    return F(len(x)) @ x

In [3]:
x = np.random.randint(-10, 10, 10)
x, dft(x)

(array([ -4, -10,   2,  -5,  -7,   0,  -5,   2,  -2,   7]),
 array([-22.        +0.00000000e+00j,   4.20820393+1.40210893e+01j,
         -6.20820393+7.80020997e+00j,  -9.20820393+1.63067181e+01j,
          7.20820393+1.92784005e+01j, -10.        -1.49406909e-14j,
          7.20820393-1.92784005e+01j,  -9.20820393-1.63067181e+01j,
         -6.20820393-7.80020997e+00j,   4.20820393-1.40210893e+01j]))

In [4]:
np.linalg.norm(dft(x) - scipy.fft.fft(x))

5.657194801626942e-14

Działa!

<br>2. Zaimplementuj również IDFT korzystając z tożsamosci [...]
Sprawdź poprawność działania funkcji realizującej DFT stosując transformację odwrotną oraz porównując uzyskane wyniki z wyjściem funkcji bibliotecznej.

In [5]:
def idft(y):
    n = len(y)
    con = np.conjugate
    return con(F(n) @ con(y))/n
y = dft(x)
idft(y)

array([-4.00000000e+00+4.26325641e-15j, -1.00000000e+01+3.90798505e-15j,
        2.00000000e+00-3.01980663e-15j, -5.00000000e+00-2.44249065e-15j,
       -7.00000000e+00-1.77635684e-16j,  2.48689958e-15-4.97379915e-15j,
       -5.00000000e+00-5.68434189e-15j,  2.00000000e+00+1.17239551e-14j,
       -2.00000000e+00+7.54951657e-16j,  7.00000000e+00-0.00000000e+00j])

In [6]:
np.linalg.norm(idft(y)-x), np.linalg.norm(idft(y)-scipy.fft.ifft(y))

(2.092758327340086e-14, 1.2689083873113277e-14)

Działa!

<br>
3. Zaimplementuj rekurencyjny algorytm Cooleya-Turkeya realizujący szybką transformację Fouriera (FFT). Porównaj szybkość jego działania z implementacją biblioteczną oraz implementacją z mnożeniem wektora przez macierz Fn dla danych
o różnym rozmiarze.

In [39]:
@lru_cache
def P(n):
    per = np.concatenate([2*np.arange(n/2), 1+2*np.arange(n/2)]).astype(np.int32)
    matrix = np.zeros((n,n))
    matrix[np.arange(n, dtype=np.int32), per] = 1
    return matrix

@lru_cache
def S(n):
    return np.diag(dzeta(n)(np.arange(n/2)))

@lru_cache
def F_rec(n):
        if n==2:
            return np.array([[1, 1], [1, -1]])
        Fh = F_rec(n//2)
        SFh = S(n) @ Fh
        return np.vstack([np.hstack([Fh, SFh]), np.hstack([Fh, -SFh])]) @ P(n)

def fft(x):
    return F_rec(len(x)) @ x

Cachuję wyniki funkcji zwracających macierze permutacji, macierze fouriera itd. - nie trzeba liczyć ich wielokrotnie.

In [40]:
x = np.random.randint(-10,10,8)
fft(x)

array([ 21.         +0.j        ,  19.07106781 +6.17157288j,
        -9.         +6.j        ,   4.92893219-11.82842712j,
       -11.         +0.j        ,   4.92893219+11.82842712j,
        -9.         -6.j        ,  19.07106781 -6.17157288j])

In [41]:
scipy.fft.fft(x)

array([ 21.         -0.j        ,  19.07106781 +6.17157288j,
        -9.         +6.j        ,   4.92893219-11.82842712j,
       -11.         -0.j        ,   4.92893219+11.82842712j,
        -9.         -6.j        ,  19.07106781 -6.17157288j])

Wygląda na to, że działa.

In [42]:
x = np.random.uniform(size=2**10)

In [43]:
%%timeit # scipy fft
y = scipy.fft.fft(x)

13.8 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [44]:
%%timeit # moje fft
y = fft(x)

550 µs ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [45]:
%%timeit # moje dft (kwadratowa złożoność)
y = dft(x)

41.2 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Implementacja scipy jest szybsza, ale nie jest źle. FFT jest oczywiście szybsze niż kwadratowe dft z punktu pierwszego.